In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import argparse 
import helper
import math

import pandas as pd
import numpy as np
import pickle
import tensorflow as tf

import sys
code_dir = './'
sys.path.append(code_dir)
import hughes_data 
import hughes_castnet_model

2023-09-19 14:34:26.087565: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-19 14:34:26.089685: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 14:34:26.135403: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-19 14:34:26.136446: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 14:34:28.750802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
class Config:
    hidden_layer_size = 32
    no_epochs = 300
    num_train_years = 7
    num_test_years = 2
    num_valid_years = 1
    window_size = None
    lead_time = None
    time_unit = 1
    group_lasso = False
    dist = None
    gl_reg_coef = None
    rnn_dropout = 0.1
    temporal_feature_size = None
    static_feature_size = None
    no_locations = None
    batch_size = 1620
    learning_rate = 0.05
    embedding_size = 32
    test_time = None
    dataset_name = None
    num_spatial_heads = None
    orthogonal_loss_coef = None

In [4]:
cook_county = False
castnet_datadir = './Data/MA/'

In [5]:
conf = Config()

parser = argparse.ArgumentParser(description='Non-Event or Event')

parser.add_argument('--ip', default='None', type=str)
parser.add_argument('--stdin', default='None', type=str)
parser.add_argument('--control', default='None', type=str)

parser.add_argument('--dataset_name', default='None', type=str)
parser.add_argument('--hidden_size', default=32, type=int)
parser.add_argument('--window_size', default=3, type=int)
parser.add_argument('--lead_time', default=2, type=int)
parser.add_argument('--gl_reg_coef', default=0.0025, type=float)
parser.add_argument('--dropout', default=0.1, type=float)
parser.add_argument('--group_lasso', default='True', type=str)
parser.add_argument('--dist', default='root_d', type=str)
parser.add_argument('--test_time', default='True', type=str)
parser.add_argument('--num_spatial_heads', default=4, type=int)
parser.add_argument('--orthogonal_loss_coef', default=0.01, type=float)

args, _ = parser.parse_known_args()

conf.dataset_name = "Chicago" if cook_county else "MA"
conf.hidden_size = args.hidden_size
conf.window_size = args.window_size
conf.lead_time = args.lead_time
conf.gl_reg_coef = args.gl_reg_coef
conf.dropout = args.dropout
conf.dist = args.dist
conf.num_spatial_heads = args.num_spatial_heads
conf.orthogonal_loss_coef = args.orthogonal_loss_coef
conf.group_lasso = helper.str2bool(args.group_lasso)
conf.test_time = helper.str2bool(args.test_time)

In [6]:
(train_svi_local, train_svi_global, train_static,
    train_sample_indices, train_dist, train_y,
    valid_svi_local, valid_svi_global, valid_static,
        valid_sample_indices, valid_dist, valid_y,
        test_svi_local, test_svi_global, test_static,
            test_sample_indices, test_dist, test_y) = hughes_data.readData(castnet_datadir, conf.dataset_name, conf.window_size, conf.lead_time,
                                                                            conf.num_train_years, conf.num_test_years, conf.num_valid_years,
                                                                            conf.dist, conf.time_unit)

num_years 14
Trimmed svi shape(1620, 14, 5)
Start indicies 0, 7, 8
num_train_years: 7
window_size: 3
lead_time: 2
Difference (num_train_years - (window_size + lead_time - 1)): 3


In [7]:
print(f'train years {train_svi_local.shape[0]/1620}')
print(f'valid years {valid_svi_local.shape[0]/1620}')
print(f'test years {test_svi_local.shape[0]/1620}')

train years 7.0
valid years 1.0
test years 2.0


In [8]:
if(conf.test_time == True):
    print("It's test time, concatting validation data to train")
    train_svi_global = np.concatenate([train_svi_global, valid_svi_global], axis=0)
    train_svi_local = np.concatenate([train_svi_local, valid_svi_local], axis=0)
    train_static = np.concatenate([train_static, valid_static], axis=0)
    train_sample_indices = np.concatenate([train_sample_indices, valid_sample_indices], axis=0)
    train_dist = np.concatenate([train_dist, valid_dist], axis=0)
    train_y = np.concatenate([train_y, valid_y], axis=0)

It's test time, concatting validation data to train


In [9]:
train_svi_global = np.swapaxes(train_svi_global, 1, 2)
valid_svi_global = np.swapaxes(valid_svi_global, 1, 2)
test_svi_global = np.swapaxes(test_svi_global, 1, 2)

temporal_feature_size = train_svi_global.shape[-1]
no_locations = train_svi_global.shape[2]
static_feature_size = train_static.shape[1]

conf.temporal_feature_size = temporal_feature_size
conf.no_locations = no_locations
conf.static_feature_size = static_feature_size

sess = tf.compat.v1.Session()

castnet = hughes_castnet_model.CASTNet(sess=sess, conf=conf)
print(train_svi_local.shape[0])

2023-09-19 14:40:56.485242: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-19 14:40:56.485290: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2023-09-19 14:40:56.485299: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: s1cmp008.pax.tufts.edu
2023-09-19 14:40:56.485438: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.104.5
2023-09-19 14:40:56.485477: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.104.5
2023-09-19 14:40:56.485485: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.104.5
/cluster/home/kheuto01/code/opioid-overdose-models/CASTNet/hughes-CASTNet/hughes_castnet_model.py:

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
12960


In [10]:
castnet.train(train_svi_local, train_svi_global, train_static, train_sample_indices, train_dist, train_y, valid_svi_local, valid_svi_global, valid_static, valid_sample_indices, valid_dist, valid_y, test_svi_local, test_svi_global, test_static, test_sample_indices, test_dist, test_y)

2023-09-19 14:40:58.934854: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled


Epoch: 1, total_loss: 66.333, regres_loss: 1.509, gl_loss: 0.145, ortho_loss: 64.678
-----------------------------------------------------------------------------
Epoch: 2, total_loss: 65.807, regres_loss: 1.105, gl_loss: 0.124, ortho_loss: 64.579
-----------------------------------------------------------------------------
Epoch: 3, total_loss: 65.59, regres_loss: 1.004, gl_loss: 0.111, ortho_loss: 64.475
-----------------------------------------------------------------------------
Iter: 1.0, total_loss: 2.616, regres_loss: 0.037, gl_loss: 0.004, ortho_loss: 2.575
Epoch: 4, total_loss: 65.11, regres_loss: 0.958, gl_loss: 0.109, ortho_loss: 64.044
-----------------------------------------------------------------------------
Epoch: 5, total_loss: 61.063, regres_loss: 0.923, gl_loss: 0.143, ortho_loss: 59.997
-----------------------------------------------------------------------------
Epoch: 6, total_loss: 47.344, regres_loss: 0.931, gl_loss: 0.204, ortho_loss: 46.208
------------------

Epoch: 44, total_loss: 16.511, regres_loss: 0.604, gl_loss: 0.454, ortho_loss: 15.452
-----------------------------------------------------------------------------
Epoch: 45, total_loss: 16.401, regres_loss: 0.503, gl_loss: 0.454, ortho_loss: 15.444
-----------------------------------------------------------------------------
Epoch: 46, total_loss: 16.328, regres_loss: 0.436, gl_loss: 0.455, ortho_loss: 15.437
-----------------------------------------------------------------------------
Iter: 15.0, total_loss: 4.557, regres_loss: 0.11, gl_loss: 0.128, ortho_loss: 4.319
Epoch: 47, total_loss: 16.284, regres_loss: 0.399, gl_loss: 0.455, ortho_loss: 15.429
-----------------------------------------------------------------------------
Epoch: 48, total_loss: 16.232, regres_loss: 0.355, gl_loss: 0.454, ortho_loss: 15.423
-----------------------------------------------------------------------------
Epoch: 49, total_loss: 16.197, regres_loss: 0.327, gl_loss: 0.454, ortho_loss: 15.416
----------

Iter: 28.0, total_loss: 2.518, regres_loss: 0.008, gl_loss: 0.065, ortho_loss: 2.445
Epoch: 88, total_loss: 15.751, regres_loss: 0.047, gl_loss: 0.405, ortho_loss: 15.298
-----------------------------------------------------------------------------
Epoch: 89, total_loss: 15.746, regres_loss: 0.046, gl_loss: 0.404, ortho_loss: 15.297
-----------------------------------------------------------------------------
Epoch: 90, total_loss: 15.743, regres_loss: 0.045, gl_loss: 0.402, ortho_loss: 15.295
-----------------------------------------------------------------------------
Iter: 29.0, total_loss: 3.151, regres_loss: 0.009, gl_loss: 0.08, ortho_loss: 3.062
Epoch: 91, total_loss: 15.741, regres_loss: 0.046, gl_loss: 0.401, ortho_loss: 15.294
-----------------------------------------------------------------------------
Epoch: 92, total_loss: 15.738, regres_loss: 0.046, gl_loss: 0.4, ortho_loss: 15.293
-----------------------------------------------------------------------------
Epoch: 93, to

Epoch: 131, total_loss: 15.647, regres_loss: 0.03, gl_loss: 0.356, ortho_loss: 15.261
-----------------------------------------------------------------------------
Iter: 42.0, total_loss: 1.253, regres_loss: 0.002, gl_loss: 0.028, ortho_loss: 1.222
Epoch: 132, total_loss: 15.643, regres_loss: 0.028, gl_loss: 0.355, ortho_loss: 15.26
-----------------------------------------------------------------------------
Epoch: 133, total_loss: 15.642, regres_loss: 0.028, gl_loss: 0.354, ortho_loss: 15.259
-----------------------------------------------------------------------------
Epoch: 134, total_loss: 15.644, regres_loss: 0.031, gl_loss: 0.354, ortho_loss: 15.259
-----------------------------------------------------------------------------
Iter: 43.0, total_loss: 1.872, regres_loss: 0.004, gl_loss: 0.042, ortho_loss: 1.827
Epoch: 135, total_loss: 15.64, regres_loss: 0.029, gl_loss: 0.353, ortho_loss: 15.258
-----------------------------------------------------------------------------
Epoch: 1

Epoch: 174, total_loss: 15.584, regres_loss: 0.019, gl_loss: 0.321, ortho_loss: 15.244
-----------------------------------------------------------------------------
Iter: 56.0, total_loss: 4.987, regres_loss: 0.006, gl_loss: 0.103, ortho_loss: 4.878
Epoch: 175, total_loss: 15.583, regres_loss: 0.019, gl_loss: 0.32, ortho_loss: 15.243
-----------------------------------------------------------------------------
Epoch: 176, total_loss: 15.583, regres_loss: 0.02, gl_loss: 0.32, ortho_loss: 15.243
-----------------------------------------------------------------------------
Epoch: 177, total_loss: 15.582, regres_loss: 0.02, gl_loss: 0.319, ortho_loss: 15.243
-----------------------------------------------------------------------------
Epoch: 178, total_loss: 15.582, regres_loss: 0.021, gl_loss: 0.319, ortho_loss: 15.243
-----------------------------------------------------------------------------
Iter: 57.0, total_loss: 0.621, regres_loss: 0.001, gl_loss: 0.013, ortho_loss: 0.608
Epoch: 17

Epoch: 217, total_loss: 15.546, regres_loss: 0.016, gl_loss: 0.294, ortho_loss: 15.236
-----------------------------------------------------------------------------
Epoch: 218, total_loss: 15.545, regres_loss: 0.015, gl_loss: 0.293, ortho_loss: 15.236
-----------------------------------------------------------------------------
Iter: 70.0, total_loss: 3.729, regres_loss: 0.003, gl_loss: 0.07, ortho_loss: 3.655
Epoch: 219, total_loss: 15.543, regres_loss: 0.014, gl_loss: 0.293, ortho_loss: 15.236
-----------------------------------------------------------------------------
Epoch: 220, total_loss: 15.564, regres_loss: 0.035, gl_loss: 0.293, ortho_loss: 15.236
-----------------------------------------------------------------------------
Epoch: 221, total_loss: 15.564, regres_loss: 0.035, gl_loss: 0.294, ortho_loss: 15.236
-----------------------------------------------------------------------------
Iter: 71.0, total_loss: 4.356, regres_loss: 0.009, gl_loss: 0.082, ortho_loss: 4.265
Epoch:

Epoch: 260, total_loss: 15.517, regres_loss: 0.016, gl_loss: 0.269, ortho_loss: 15.232
-----------------------------------------------------------------------------
Epoch: 261, total_loss: 15.516, regres_loss: 0.016, gl_loss: 0.268, ortho_loss: 15.232
-----------------------------------------------------------------------------
Epoch: 262, total_loss: 15.515, regres_loss: 0.015, gl_loss: 0.267, ortho_loss: 15.232
-----------------------------------------------------------------------------
Iter: 84.0, total_loss: 2.487, regres_loss: 0.002, gl_loss: 0.043, ortho_loss: 2.442
Epoch: 263, total_loss: 15.514, regres_loss: 0.015, gl_loss: 0.267, ortho_loss: 15.232
-----------------------------------------------------------------------------
Epoch: 264, total_loss: 15.514, regres_loss: 0.015, gl_loss: 0.266, ortho_loss: 15.232
-----------------------------------------------------------------------------
Epoch: 265, total_loss: 15.512, regres_loss: 0.014, gl_loss: 0.266, ortho_loss: 15.232
---

In [13]:
model_save_path = os.path.join('./', 'Results')
    
if cook_county:
    model_save_path = os.path.join(model_save_path, 'Chicago')
else:
    model_save_path = os.path.join(model_save_path, 'MA')

castnet.save(model_save_path)
castnet.save_results()